In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import sys
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import io

In [4]:
fpath = "/scratch/indikar_root/indikar1/cstansbu/scpc_test/align_table/o1b47.GRCm39.raw.parquet"

df = pd.read_parquet(fpath)
print(f"{df.shape=}")
df.head()

df.shape=(2693962, 15)


,read_name,fragment_index,is_digested_fragment,align_type,is_forward,is_mapped,mean_align_base_quality,read_length,read_start,read_end,chrom,reference_start,reference_end,mapping_quality,alignment_score
0,906ccb24-ac70-4b13-94de-b88bb2b28d28,-1,False,secondary,True,True,-1.0,0,124,1652,1,3092614,3094149.0,0,1900
1,aad3c6f1-80b0-45d9-beb8-f64b68a3a00f,-1,False,secondary,True,True,-1.0,0,113,1464,1,3092616,3093982.0,0,1556
2,fc10433d-2301-4f5d-bfc8-5db37c57c276,-1,False,secondary,True,True,-1.0,0,116,1642,1,3092616,3094149.0,0,1938
3,8fc65884-0806-4ac5-9dde-f76b7594364b,-1,False,secondary,False,True,-1.0,0,96,1599,1,3092616,3094149.0,0,1556
4,8472942d-3796-44ea-8449-4055c0f65002,-1,False,secondary,True,True,-1.0,0,114,474,1,3137393,3137748.0,0,496


In [12]:
# plot a read

read_name = np.random.choice(df['read_name'], 1)[0]
print(f"{read_name=}")

pdf = df[df['read_name'] == read_name]
pdf.head

read_name='01d79c87-973d-40e1-8347-f5f5993a0772'


<bound method NDFrame.head of                                     read_name  fragment_index  \
87267    01d79c87-973d-40e1-8347-f5f5993a0772              -1   
883137   01d79c87-973d-40e1-8347-f5f5993a0772              -1   
998264   01d79c87-973d-40e1-8347-f5f5993a0772              -1   
1004391  01d79c87-973d-40e1-8347-f5f5993a0772              -1   
1009960  01d79c87-973d-40e1-8347-f5f5993a0772              -1   
1016007  01d79c87-973d-40e1-8347-f5f5993a0772              -1   
1021624  01d79c87-973d-40e1-8347-f5f5993a0772              -1   
1027834  01d79c87-973d-40e1-8347-f5f5993a0772              -1   
1058309  01d79c87-973d-40e1-8347-f5f5993a0772              -1   
1141277  01d79c87-973d-40e1-8347-f5f5993a0772              -1   
1234845  01d79c87-973d-40e1-8347-f5f5993a0772              -1   
2116598  01d79c87-973d-40e1-8347-f5f5993a0772              -1   

         is_digested_fragment     align_type  is_forward  is_mapped  \
87267                   False  supplementary       Fa

In [6]:
# plot a set of reads

mapq = 60
chrom = '2'
sample = 5

pdf = df.copy()

pdf = pdf[pdf['mapping_quality'] == mapq]
pdf = pdf[pdf['chrom'] == chrom]

print(f"{pdf.shape=}")
pdf.head()

pdf.shape=(101391, 15)


,read_name,fragment_index,is_digested_fragment,align_type,is_forward,is_mapped,mean_align_base_quality,read_length,read_start,read_end,chrom,reference_start,reference_end,mapping_quality,alignment_score
143741,6d6ca55d-3799-464f-bd4b-7e2aee5faa40,-1,False,supplementary,True,True,16.665584,308,0,308,2,3789528,3789838.0,60,562
143742,fded5ca6-325c-40be-8849-fdc8f33401be,-1,False,supplementary,True,True,16.646302,311,0,311,2,3789528,3789838.0,60,602
143743,009d704b-4368-43f6-b265-0f13e16c284d,-1,False,supplementary,False,True,16.503226,310,0,310,2,3789528,3789838.0,60,578
143744,1d1f27b2-5eb5-4a83-bf39-e3c8a09094f3,-1,False,supplementary,False,True,16.818653,193,0,193,2,3831030,3831221.0,60,336
143745,1837a408-2c0f-4176-841e-0759afab8bd5,-1,False,supplementary,True,True,16.000000,92,0,92,2,3854051,3854144.0,60,178
